#  Machine Learning - Mini Project II

### Learning Algorithm: Classifier

Created on March 10, 2019 by Diogo Cosin <d.ayresdeoliveira@jacobs-university.de> and Ralph Florent <r.florent@jacobs-university.de>.

### Description
Train a classifier for the Digits dataset by implementing a full processing pipeline from feature extraction to (linear) classifier training, attempting to squeeze performance out of the classifier using cross-validation and regularization techniques.

### Summary
The script below is intended to... 

WIP

Note: The algorithm is tested on the OCR datasets from the `DigitsBasicsRoutine.zip`, which was provided by Professor Dr. H. Jaeger, Machine Learning Professor at [Jacobs University Bremen](https://www.jacobs-university.de).

In [14]:
""" Learning Algorithm: Classifier """

# -*- coding: utf-8 -*-
# 
# Created on April 01, 2019
# Authors: 
#        Diogo Cosin <d.ayresdeoliveira@jacobs-university.de>,
#        Ralph Florent <r.florent@jacobs-university.de>


# Import relevant libraries
import sys
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from timeit import default_timer as timer
np.set_printoptions(threshold=sys.maxsize)

sys.path.append('./assets/')
from miniprojectone import get_data_points, k_means, get_codebooks, assign_randomly

TOTAL_DIGIT_CLASSES = 10

# START: Helper functions
def load_data(one_hot_encoding=None):
    """ Load the data and parse (if specified, one-hot encoding) numbered 
        class labels.
        
        Parameters
        ----------
        one_hot_encoding: bool, None
            determine whether one-hot encoding should be used or not
            
        Returns
        -------
        dataframe: array-like (n_samples, m_features)
    """
    # load data frame with no class labels defined
    dataframe = get_data_points()
    if one_hot_encoding is None:
        return dataframe
    return inject_label(dataframe, one_hot_encoding)

def one_hot_encode(ith, k=TOTAL_DIGIT_CLASSES):
    """ Apply one-hot encoding technique for a k-dimensional vector
        
        This function is intended to work like a lightweight version one-hot-encoder.
        It is adapted to our specific needs. 
        
        Parameters
        ----------
        ith: int
            ith position of the one-of-K discrete label
        
        k: int
            number of k-classes for the encoding vector
            
        Returns
        -------
        encoded: array
            vector with zeros and one in the ith position
    """
    if ith > k:
        ith = k # avoid out of bound exception
        
    index = ith - 1
    encoded = np.zeros(k)
    encoded[index] = 1
    return encoded


def inject_label(dataset, one_hot_encoding=False):
    """ Inject into data frame class labels as numbered or one-hot encoded 
    """
    dataframe, k_class = [], TOTAL_DIGIT_CLASSES
    digits = np.array_split(dataset, k_class) # split into 10 arrays
    for i in range(k_class):
        digit =  digits[i] # n-obs x k-dim
        if i == 0: 
            i = 10 # define "0" as class 10
        encoded = one_hot_encode(i) if one_hot_encoding else i
        
        for point in digit:
            dataframe.append( np.insert(point, len(point), encoded) )
            
    return np.array(dataframe) 


def split_data(dataframe):
    """ Split data frame into training and testing data
    
    Parameters
    ----------
    dataframe: array-like (n_samples, m_features)
        2000 digit patterns x 240 features and 10 one-hot encoded class labels

    Returns
    -------
    encoded: list
        list containing array of training and array of testing data
    """
    digits = np.array_split(dataframe, 10)
    
    train_data, test_data = [], []

    for digit in digits:
        train_data.extend(digit[:100])
        test_data.extend(digit[100:])
    
    return [np.array(train_data), np.array(test_data)]


def select_features(dataset, codebooks):
    """ Apply Euclidean distance between k-means centroid and each point
    to extract k features.
    """
    features = []
    # transfrom xi -> fi by reducing dimensionality from n to k features
    for point in dataset:
        # compute distance between data point and centroid 
        feature = [np.linalg.norm(point - c) for c in codebooks]
        feature.append(1) # padded with a trailing 1 
        features.append( np.array(feature) )
    
    return np.array(features)

def compute_linreg(Phi, Zi, alpha=0):
    """ Algorithm to compute weights of the matrix W that solves the linear regression
    
    Data: Given m-dimensional feature vectors (fi) and k-dimensional target vectors (zi)
    Result: a kxm-dimensional weight matrix Wopt that solves the linear regression
    
    1) Sort the feature vectors row-wise into an Nxm matrix (Phi) and the targets into an
    Nxk matrix (Z)
    2) Compute the pseudo inverse of Phi x Zi as the Wopt transposed.
    
    The pseudo-inverse of a matrix A, denoted A^+, is defined as: “the matrix that ‘solves’ 
    [the least-squares problem] Ax = b,” i.e., if x is said solution, then A^+ is 
    that matrix such that x = A^+b.
    
    ref: 
        https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.pinv.html
        https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html
        https://docs.scipy.org/doc/numpy/reference/generated/numpy.identity.html
    """
    # compute rigde regression factor 
    ridge_factor = alpha * np.identity(Phi.shape[1]) # Imxm identity matrix
    # compute pseudo inverse of Phi x Zi
    Wopt = np.linalg.inv(np.dot(Phi.T, Phi) + ridge_factor).dot(Phi.T).dot(Zi)
    return Wopt.T


def check_accuracy(Y_ground, Y_pred):
    # Y_ground and Y_pred are N by k matrices
    
    ground_labels = np.argmax(Y_ground, axis=1) # N-dimensional vector with the ground labels
    predicted_labels = np.argmax(Y_pred, axis=1) # N-dimensional vector with the highest probability position
    
    correct_classifications = sum(predicted_labels == ground_labels)
    accuracy = float(correct_classifications)/Y_ground.shape[0]
    
    return accuracy


def check_MSE(Y_ground, Y_pred):
    # Y_ground and Y_pred are N by k matrices
    
    Y_norm = np.linalg.norm(Y_ground - Y_pred, axis=1) # N-dimensional vector with the norms
    MSE = np.square(Y_norm).mean()
    
    return MSE


def predict_hypothesis_matrix(model, X):
    hypothesis_matrix = model@X.T # result in a k by N matrix
    return hypothesis_matrix.T # return N by k hypothesis matrix


def k_fold_cv(dataset, nbr_folds=5, k=10, alpha=0.0):
    # copy the data and shuffle it
    data = dataset.copy()
    
    # divide into k-groups
    folds = assign_randomly(data, nbr_folds)
    
    accuracy_train, accuracy_test = [], []
    MSE_train, MSE_test = [], []
    
    # train models for the folds
    for i in range(nbr_folds):
#         print("Processing fold {}".format(i))
        
        # distribute train and test data
        test_data, train_data = folds[i], []
        for j in range(nbr_folds):
            if i != j:
                train_data.extend(folds[j])
        
        train_data = np.array(train_data)
        test_data = np.array(test_data)
                
        # separate X (inputs) and Y (labels) for training and testing data
        x_train, y_train = train_data[:, :-TOTAL_DIGIT_CLASSES], train_data[:, -TOTAL_DIGIT_CLASSES:]
        x_test, y_test = test_data[:, :-TOTAL_DIGIT_CLASSES], test_data[:, -TOTAL_DIGIT_CLASSES:]
        
        # K-mean dimensionality reduction
        start = timer()
        clusters = k_means(x_train, k)
        codebooks = get_codebooks(clusters)
        end = timer()
        print("--- K-means elapsed time: {:1.10f}".format(end - start))
        start = timer()
        x_train = select_features(x_train, codebooks)
        x_test = select_features(x_test, codebooks)
        end = timer()
        print("--- Feature extraction elapsed time: {:1.10f}".format(end - start))
        
        # compute models for training data
        model = compute_linreg(x_train, y_train, alpha)
        
        # prediction on training and testing data
        pred_train = predict_hypothesis_matrix(model, x_train)
        pred_test = predict_hypothesis_matrix(model, x_test) 
        
        # assess accuracy
        _acc_train = check_accuracy(pred_train, y_train)
        _acc_test  = check_accuracy(pred_test, y_test)
        
        accuracy_train.append(_acc_train)
        accuracy_test.append(_acc_test)

        # assesss MSE
        _MSE_train = check_MSE(pred_train, y_train)
        _MSE_test = check_MSE(pred_test, y_test)
        
        MSE_train.append(_MSE_train)
        MSE_test.append(_MSE_test)
        
        cv_accuracy_train = np.mean(accuracy_train)
        cv_accuracy_test = np.mean(accuracy_test)
        cv_MSE_train = np.mean(MSE_train)
        cv_MSE_test = np.mean(MSE_test)
        
    return cv_accuracy_train, cv_accuracy_test, cv_MSE_train, cv_MSE_test

### Loading training and testing data
The `load_data` function helps framing the data with numbered or one-hot encoding class labels to build an in-memory data frame. 

Given this data frame, let's load the first 100 digit-patterns as training data and the second 100 as testing data, for a specific digit.

In [2]:
dataframe = load_data(one_hot_encoding=True)

# Distribute dataframe into training and testing data
training_data, testing_data = split_data(dataframe)

print(dataframe.shape, training_data.shape, testing_data.shape)

(2000, 250) (1000, 250) (1000, 250)


### Feature selection

Let's use K-means clustering algorithm to select relevant features in the training data

In [15]:
raw_dataframe = load_data()
raw_training_data, raw_testing_data = split_data(raw_dataframe)

clusters = k_means(raw_training_data, 10)
codebooks = get_codebooks(clusters)
selected_features = select_features(raw_training_data, codebooks)

print(raw_training_data.shape, selected_features.shape)

(1000, 240) (1000, 11)


Inject now the one hot encoded labels in the reduced data.

In [16]:
features_labels = inject_label(selected_features, one_hot_encoding=True)
features_labels.shape

(1000, 21)

### Classifier: Linear Regression

Let's compute the linear regression weights with a trailing 1 as bias



In [21]:
Phi = features_labels[:,:-TOTAL_DIGIT_CLASSES]
Zi = features_labels[:,-TOTAL_DIGIT_CLASSES:]
weights = compute_linreg(Phi, Zi)
weights.shape

(10, 11)

Now we check the accuracy and MSE through 10-fold cross-validation. In this case, we need reload the data, but now with one hot encoded labels.

In [29]:
raw_dataframe = load_data(one_hot_encoding=True)
raw_training_data, raw_testing_data = split_data(raw_dataframe)

In [32]:
nbr_folds = 5
k = 20
alpha = 0.0

cv_accuracy_train, cv_accuracy_test, cv_MSE_train, cv_MSE_test = k_fold_cv(raw_training_data, nbr_folds, k, alpha)

Processing fold 0
--- K-means elapsed time: 12.2034717980
--- Feature extraction elapsed time: 0.1785829620
Processing fold 1
--- K-means elapsed time: 14.0005034330
--- Feature extraction elapsed time: 0.1811089210
Processing fold 2
--- K-means elapsed time: 14.9120720690
--- Feature extraction elapsed time: 0.1804820530
Processing fold 3
--- K-means elapsed time: 10.6060940190
--- Feature extraction elapsed time: 0.1773576660
Processing fold 4
--- K-means elapsed time: 13.6522932980
--- Feature extraction elapsed time: 0.1755237510


In [33]:
print("The train accuracy is {:1.2f}%".format(100*cv_accuracy_train))
print("The test accuracy is {:1.2f}%".format(100*cv_accuracy_test))
print("The train MSE is {:1.4f}".format(cv_MSE_train))
print("The test MSE is {:1.4f}".format(cv_MSE_test))

The train accuracy is 94.45%
The test accuracy is 92.70%
The train MSE is 0.2290
The test MSE is 0.2517


## Grid Searching Alpha and K Optimal Values

We now try different combinations of alpha and number of K-means clusters, checking the metrics for each combination. The objective here is to optimize the classifier improving, this way, the performance metrics.

The results are stored in the dictionary `grid_search_metrics`.

In [5]:
alpha_list = np.arange(0.0, 1.01, 0.1)
k_list = [10, 20]

In [11]:
alpha_list = [0.0]
k_list = [2, 5, 10, 20, 50, 100, 200, 400, 800]

In [12]:
k_fold = 10 

grid_search_metrics = {}
grid_search_metrics["alpha"] = []
grid_search_metrics["K"] = []
grid_search_metrics["cv_acc_train"] = []
grid_search_metrics["cv_acc_test"] = []
grid_search_metrics["cv_MSE_train"] = []
grid_search_metrics["cv_MSE_test"] = []

for k in k_list:
    print("=> Processing k={}".format(k))
#     clusters = k_means(raw_training_data, k)
#     codebooks = get_codebooks(clusters)
#     selected_features = select_features(raw_training_data, coodebooks)
#     features_labels = inject_label(selected_features, one_hot_encoding=True)
    
    for alpha in alpha_list:
#         print("=====> Processing alpha={}".format(alpha))
        grid_search_metrics["alpha"].append(alpha)
        grid_search_metrics["K"].append(k)
        
        cv_accuracy_train, cv_accuracy_test, cv_MSE_train, cv_MSE_test = k_fold_cv(raw_training_data, k_fold, k, alpha)
        
        grid_search_metrics["cv_acc_train"].append(cv_accuracy_train)
        grid_search_metrics["cv_acc_test"].append(cv_accuracy_test)
        grid_search_metrics["cv_MSE_train"].append(cv_MSE_train)
        grid_search_metrics["cv_MSE_test"].append(cv_MSE_test)

=> Processing k=2
Processing fold 0
--- K-means elapsed time: 3.6661977560
--- Feature extraction elapsed time: 0.0296250910
Processing fold 1
--- K-means elapsed time: 4.8218664210
--- Feature extraction elapsed time: 0.0402324230
Processing fold 2
--- K-means elapsed time: 4.9351109660
--- Feature extraction elapsed time: 0.0401054670
Processing fold 3
--- K-means elapsed time: 10.2019201340
--- Feature extraction elapsed time: 0.0407860550
Processing fold 4
--- K-means elapsed time: 4.1424964850
--- Feature extraction elapsed time: 0.0378704560
Processing fold 5
--- K-means elapsed time: 3.5853394060
--- Feature extraction elapsed time: 0.0400042550
Processing fold 6
--- K-means elapsed time: 3.1725882070
--- Feature extraction elapsed time: 0.0396977290
Processing fold 7
--- K-means elapsed time: 4.3883294160
--- Feature extraction elapsed time: 0.0402389380
Processing fold 8
--- K-means elapsed time: 6.3378127710
--- Feature extraction elapsed time: 0.0368031900
Processing fold 9


/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--- K-means elapsed time: 19.5398337420
--- Feature extraction elapsed time: 0.3625948350
Processing fold 6
--- K-means elapsed time: 25.4884283120
--- Feature extraction elapsed time: 0.4249371170
Processing fold 7
--- K-means elapsed time: 19.8938471070
--- Feature extraction elapsed time: 0.5017061660
Processing fold 8
--- K-means elapsed time: 28.2417788210
--- Feature extraction elapsed time: 0.4322608820
Processing fold 9
--- K-means elapsed time: 13.4851431900
--- Feature extraction elapsed time: 0.4128898300
=> Processing k=100
Processing fold 0
--- K-means elapsed time: 25.5268676080
--- Feature extraction elapsed time: 0.7289269280
Processing fold 1
--- K-means elapsed time: 27.3092490770
--- Feature extraction elapsed time: 0.7781772930
Processing fold 2
--- K-means elapsed time: 36.0151101330
--- Feature extraction elapsed time: 1.0685135580
Processing fold 3
--- K-means elapsed time: 37.6765251140
--- Feature extraction elapsed time: 0.7388889260
Processing fold 4
--- K-me

In [9]:
pd.DataFrame(grid_search_metrics)

,alpha,K,cv_acc_train,cv_acc_test,cv_MSE_train,cv_MSE_test
0,0.0,10,0.844222,0.847,0.375235,0.389977
1,0.1,10,0.833556,0.838,0.397678,0.408196
2,0.2,10,0.839667,0.808,0.377793,0.400422
3,0.3,10,0.837667,0.826,0.395476,0.408668
4,0.4,10,0.836111,0.837,0.384749,0.396459
5,0.5,10,0.838333,0.850,0.385792,0.384418
6,0.6,10,0.830667,0.811,0.391607,0.424242
7,0.7,10,0.838222,0.837,0.384801,0.398354
8,0.8,10,0.822000,0.817,0.397297,0.414552
9,0.9,10,0.817222,0.799,0.394618,0.418543


In [13]:
pd.DataFrame(grid_search_metrics)

,alpha,K,cv_acc_train,cv_acc_test,cv_MSE_train,cv_MSE_test
0,0.0,2,0.363778,0.359,0.767940,0.772451
1,0.0,5,0.663222,0.647,0.612294,0.618742
2,0.0,10,0.827333,0.827,0.402264,0.409435
3,0.0,20,0.947222,0.941,0.227399,0.248641
4,0.0,50,0.967778,0.965,0.159679,0.189419
5,0.0,100,0.979667,0.970,0.119638,0.157589
6,0.0,200,0.987889,0.970,0.084664,0.135771
7,0.0,400,0.996111,0.976,0.049026,0.120286
8,0.0,800,1.000000,0.982,0.006403,0.105101
